Chapter 5 - Lab on The Finite Element Method
============================================

# Exercise I

Let $f\in L^{2}(0,1)$.

Consider

$$ (5.1)\quad\left\{
    \begin{array}{ll}
       - \displaystyle u^{\prime\prime}=f \quad \textrm{in}\ ]0,1[,\\
       u(0)=\alpha\quad\textrm{and}\quad u(1)=\beta. 
    \end{array}\right.$$
                                         
with $\alpha,\beta\in\mathbb{R}$.

In this notebook, we will need several librairies:

In [1]:
# Two standard librairies for mathematical functions

import math
import numpy as np


# A standard library for plotting

import matplotlib.pyplot as plt


# Functions for numerical quadrature
# (numerical approxmation of an integral)

from scipy.integrate import quad
from scipy import special